In [1]:
import numpy as np
import pandas as pd

from keras.models import load_model

In [2]:
df = pd.read_parquet('test.parquet')


In [ ]:
df['values'].iloc[361]
# test dataset has missing values or read_parquet reads it incorrectly

In [4]:
dftrain = pd.read_parquet('train.parquet')
all_period = dftrain['dates'][30]
correct_data = []
for i in range(20000):
    start = df['dates'].iloc[i][0]
    for j in range(len(all_period)):
        if all_period[j] == start:
            correct_data.append(np.concatenate((np.zeros(j) , df["values"].iloc[i] , np.zeros(97-j-len(df['dates'].iloc[i])))))
df['correct_values'] = correct_data

In [ ]:
missing_indices = df.index[df['correct_values'].apply(lambda x: np.isnan(x).any())].tolist()
for i in missing_indices:
    df['correct_values'].iloc[i] = np.zeros(97)

In [6]:
model = load_model("model2.keras")

In [7]:
df['score'] = model.predict(np.stack(df['correct_values']))

625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step


In [ ]:
df = df.drop(['dates', 'values', 'correct_values'], axis=1)
df.head()

,id,score
0,6125,0.144497
1,26781,0.089150
2,13333,0.305250
3,53218,0.011847
4,84204,0.570568


In [16]:
df.to_csv('submission.csv')